In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten


/home/fede/Documentos/Machine-learning/PT-1/tp1_exploration/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
import cv2

def load_images(path):
    img_data = []
    labels = []
    idx_to_label = []
    i = -1
    for fruit in os.listdir(path):
        fruit_path = os.path.join(path,fruit)
        labels.append(fruit)
        i = i+1
        for img in os.listdir(fruit_path):
            img_path = os.path.join(fruit_path,img)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (100, 100))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            img_data.append(image)
            idx_to_label.append(i)
    return np.array(img_data),np.array(idx_to_label),labels

In [24]:
training_data_path = '/home/fede/Documentos/Machine-learning/PT-1/data/Training'
validation_data_path = '/home/fede/Documentos/Machine-learning/PT-1/data/Validation'
X_train,y_train,label_data = load_images(training_data_path)
print(X_train.shape)
print(y_train.shape)
print(label_data)
X_test,y_test,label_data_garbage = load_images(validation_data_path)

(33657, 64, 64, 3)
(33657,)
['Litchi', 'Strawberry', 'Grape White 2', 'Cherry 1', 'Banana', 'Apple Red Yellow', 'Kiwi', 'Cactus fruit', 'Banana Red', 'Avocado', 'Kaki', 'Lemon Meyer', 'Kumquats', 'Mandarine', 'Papaya', 'Pineapple Mini', 'Cocos', 'Raspberry', 'Pear Abate', 'Grape White', 'Peach Flat', 'Apple Golden 3', 'Apple Red 1', 'Apricot', 'Pear Monster', 'Granadilla', 'Melon Piel de Sapo', 'Grapefruit White', 'Apple Golden 1', 'Tangelo', 'Pitahaya Red', 'Cantaloupe 1', 'Cantaloupe 2', 'Peach', 'Cherry Rainier', 'Apple Granny Smith', 'Pomegranate', 'Huckleberry', 'Guava', 'Grape Pink', 'Pepino', 'Limes', 'Dates', 'Carambula', 'Orange', 'Apple Golden 2', 'Clementine', 'Pear', 'Apple Red 2', 'Tamarillo', 'Plum', 'Avocado ripe', 'Apple Red Delicious', 'Apple Braeburn', 'Strawberry Wedge', 'Apple Red 3', 'Quince', 'Mango', 'Pear Williams', 'Nectarine', 'Salak', 'Lemon', 'Cherry 2', 'Pineapple', 'Passion Fruit', 'Grapefruit Pink', 'Maracuja']
